In [2]:
#Import Libraries
import os
import json
import csv
import matplotlib.pyplot as plt
import sklearn
import tensorflow
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import torch
import torch.nn as nn
import time
import spacy
import kagglehub

#NLP Packages
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from nltk.corpus import stopwords
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from IPython.display import display, clear_output

import pyarrow


2025-02-25 13:19:17.936956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740518358.233836   33558 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740518358.291418   33558 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 13:19:18.976701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/abhaydesktop/Documents/Repo/abhayma1000/DS3010FinalProject/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarni

# Load in and organize data

In [ ]:
# import pandas as pd

# # Define a generator to read the parquet file in chunks
# def read_parquet_in_chunks(file_path, chunksize=10000):
#     for chunk in pd.read_parquet(file_path, engine='pyarrow', columns=None):
#         yield chunk

# # Initialize an empty dataframe
# df_rb = pd.DataFrame()

# # Read the parquet file in chunks and concatenate them
# file_path = os.path.join('data', 'reviews.parquet')
# for chunk in read_parquet_in_chunks(file_path):
#     df_rb = pd.concat([df_rb, chunk], ignore_index=True)


In [3]:
import dask.dataframe as dd

save_folder = os.path.join('data', 'reviews_parquets_2')

# Load all Parquet files as a Dask DataFrame (lazy loading)
print("Loading from folder: ", save_folder)
df_dask = dd.read_parquet(os.path.join(save_folder, "*.parquet"), engine="pyarrow")

display(df_dask.npartitions)

display(df_dask.head(5))

# for partition in df_dask.to_delayed():
#     partition_df = partition.compute()  # Load only one partition into memory
#     print(partition_df.shape)  # Process chunk-by-chunk

Loading from folder:  data/reviews_parquets_2


7

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,qDVD1-9K8PYkYQ_NjhDfbg,_DClBUD4baUoqrMv_9mC4A,U3grYFIeu6RgAAQgdriHww,4,0,0,0,good need bother loud noise bar restaurant goo...,2018-03-19 15:46:11,Brophy Bros - Santa Barbara,...,CA,93109,34.403759,-119.693992,4.0,2940,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Cocktail Bars, Nightlife, Seafood, Restaurant...","{'Friday': '11:0-16:0', 'Monday': '0:0-0:0', '..."
1,ei6N302RZvPLERM6rmIKrw,H2Hri23uglssz0_YdoXwig,3JpJ3b8r5jMdAb1yPmchrQ,5,0,0,0,fry chicken course go greasy also well hot chi...,2014-09-15 04:04:07,Prince's Hot Chicken Shack,...,TN,37207,36.230037,-86.761003,4.0,907,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, American (Traditional), Southern]",None
2,glPRhDcijNDdBXuYt3o8YQ,P-o2T0qiHCLnza4kx5bsGw,QcMIUKZr9s6LFy7jsDvNVw,5,0,0,0,amazing brunch sunday brandywine prime breakfa...,2017-02-20 14:47:48,Brandywine Prime,...,PA,19317,39.872561,-75.590590,4.0,195,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Nightlife, Bars, Steakhouses, Burgers, Restau...","{'Friday': '16:30-21:30', 'Monday': None, 'Sat..."
3,_8HrThlckhRGV2lombOL4Q,guIb53iyjk62zx05cmemTw,Bj7DL3SwmYHZYqPduAsa2w,5,1,0,0,go lunch bomb order alligator sausage po boy f...,2018-12-30 01:24:33,Johnny's Po-Boys,...,LA,70130,29.955405,-90.064477,3.5,1451,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Breakfast & Brunch, Restaurants, Sandwiches]","{'Friday': '8:0-16:30', 'Monday': '8:0-16:30',..."
4,g8Y3nnLDyDxWcJ6IYyHvew,rmo1L9DoDc_BRkzsS7M4Ug,KFFoKuUnGry4GyV2qiRwKQ,2,2,0,0,food okay item hot staff great wait time ridic...,2017-04-02 04:30:26,Riverview Restaurant & Bar,...,NJ,08016,40.080370,-74.858910,3.5,111,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Nightlife, Restaurants, Bars, American (New),...","{'Friday': '15:0-22:0', 'Monday': None, 'Satur..."


In [4]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [5]:
#Create inputs (review text) and outputs (categories)
inputs = df_dask["text"].compute().values

#One-Hot Encode
mlb = MultiLabelBinarizer()
outputs = mlb.fit_transform(df_dask['categories'].compute()).astype(float)

#Total Number of Unique Labels
num_labels = outputs.shape[1]

In [ ]:
#Make sure Tokenizer is Working
print('\nOriginal: {}\n'.format(inputs[0]))

# Print the sentence split into tokens.
print('Tokenized: {}\n'.format(tokenizer.tokenize(inputs[0])))

# Print the sentence mapped to token ids.
print('Token IDs: {}'.format(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inputs[0]))))

In [ ]:
input_ids = []
attention_masks = []

print('Start Run')
start_time = time.time()

# For every review...
for rev in inputs:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        rev,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        padding = 'max_length',           
                        max_length = 200,           # Pad & truncate all sentences. Reviews over 512 so truncate there for now.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True        # Truncate at max_length
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tokenizer Encoding: {elapsed_time:.4f} seconds")

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
outputs = torch.tensor(outputs).clone().detach()

# Print sentence 0, now as a list of IDs.
print('Original: ', inputs[0])
print('Token IDs:', input_ids[0])
print('Done')

In [ ]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, outputs)

# Create a 75-15-10 train-validation-test split.

# Calculate the number of samples to include in each set.
train_size = int(0.75 * len(dataset))
val_size_b4_test = int(len(dataset) - train_size)

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size_b4_test])

val_size = int(0.15 * len(dataset))
test_size = int(len(val_dataset)-val_size)
val_dataset, test_dataset = random_split(val_dataset, [val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# Train the model

In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = num_labels, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    problem_type = "multi_label_classification" # Defaults loss function to BCEWithLogitsLoss
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Define Loss function Compatible with Multi-label classification may be redundant givent that "problem_type" specified as multi-label-classification in model
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
###########################
# Train with training set #
###########################
def train(model, iterator, optimizer, criterion, device, scheduler, epoch):
    
    # Enter Train Mode
    model.train()
    train_loss = 0

    #Number of iterations equal to total train dataset / batch size
    for step, batch in enumerate(iterator):
        #Print progress in epoch
        print(f"Progress: {step+1}/{len(iterator)}", end='\r')
        # Parse iterator tensor dataset for important information
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        # Generate prediction
        optimizer.zero_grad()
        
        # Compute gradients and update weights
        loss = criterion(output.logits, b_labels) # BCEWithLogitsLoss has sigmoid
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # accumulate train loss
        train_loss += loss
    
    # print completed result
    print()
    print('Train Loss: %f' % (train_loss))
    return train_loss

#############################
# Validate with testing set #
#############################
def test(model, iterator, optimizer, criterion, device, epoch):

    # Enter Evaluation Mode
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for step, batch in enumerate(iterator):
        
            #Print progress in epoch
            print(f"Progress: {step+1}/{len(iterator)}", end='\r')
            print(f'Time per iteration: {datetime.timedelta(seconds=(time.time() - start_time)//(step+1))}', end='\r')
            
            # Parse iterator tensor dataset for important information
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # generate prediction
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            prob = output.logits.sigmoid()   # BCEWithLogitsLoss has sigmoid
            
            # record processed data count
            total += (b_labels.size(0)*b_labels.size(1))

            # take the index of the highest prob as prediction output
            THRESHOLD = 0.7
            prediction = prob.detach().clone()
            prediction[prediction > THRESHOLD] = 1
            prediction[prediction <= THRESHOLD] = 0
            correct += prediction.eq(b_labels).sum().item()

        print()
    
    #print completed result
    acc = 100.*correct/total
    print('Correct: %i  / Total: %i / Test Accuracy: %f' % (correct, total, acc))
    return acc

In [ ]:
for e in range(epochs):
    
    print(f"\n===== Epoch {e+1}/{epochs} =====")
    
    # training
    print("Training started ...")
    train(model, train_dataloader, optimizer, criterion, device, scheduler, e)

    # validation testing
    print("Testing started ...")
    test(model, validation_dataloader, optimizer, criterion, device, e)

In [ ]:
import smtplib
from email.mime.text import MIMEText

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail, change if using Outlook or another service
SMTP_PORT = 465  # SSL port (or use 587 for TLS)
SENDER_EMAIL = "abhaymathur1000@gmail.com"  # Replace with your email
SENDER_PASSWORD = "blom axpb quot zcdv"  # Use the generated app password
RECEIVER_EMAIL = "abhaymathur1000@gmail.com"  # Your email (or another recipient)

def send_email(subject, body=""):
    """Sends an email notification"""
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")



# Notify when the script completes
# send_email("✅ Script Completed", "Your Python script has finished running.")
send_email("Training complete", "Training complete")

# Save Model Code

In [ ]:
save_path = os.path.join('models', 'berttrained')

count = 0
while os.path.exists(save_path + str(count)):
    count += 1

model.save_pretrained(save_path + str(count))

# Test model

In [6]:
model = BertForSequenceClassification.from_pretrained(os.path.join('models', 'berttrained' + str(0)))

In [7]:
acc = test(model, test_dataloader, optimizer, criterion, device, e)
print("Test model accuracy: ", acc)

NameError: name 'test' is not defined

In [ ]:
#Will have to adjust this but not sure how to do so yet without the model - save for later
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            
            predictions.extend(list(pred_flat))

In [ ]:
for i in predictions:
    vec = np.zeros((num_labels))
    vec[int(i)] = 1
    # print(vec.shape)
    print(mlb.inverse_transform(vec.reshape(1, -1)))

In [ ]:
len(mlb.classes_)